In [ ]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2024 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

import os
import time
from enum import Enum
import pandas as pd
import numpy as np

import pyomo.environ as pyo
from pyomo.common.fileutils import this_file_dir
from pyomo.util.calc_var_value import calculate_variable_from_constraint

import idaes.core.util.scaling as iscale
from idaes.core.solvers import get_solver
import idaes.logger as idaeslog
import idaes.core.util.model_serializer as ms
from idaes.core.util.model_statistics import degrees_of_freedom as dof
from idaes_examples.mod.power_gen.soc_dynamic_flowsheet import SocStandaloneFlowsheet as SocFlowsheet
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from idaes.models.control.controller import ControllerType, ControllerMVBoundType, ControllerAntiwindupType
# from save_results import save_results
from idaes.models.properties import iapws95
from idaes.core.util.model_statistics import degrees_of_freedom as dof
from IPython.display import SVG, display

# SOC Flowsheet --- Steady State Optimization
Author: Douglas Allan
Maintainer: Douglas Allan  
Updated: 2024-23-04

## 1. Introduction

TODO 

## 2. Model Description

TODO

In [ ]:
class OperatingScenario(Enum):
    maximum_production = 1
    minimum_production = 2
    power_mode = 3
    neutral = 4

def scale_indexed_constraint(con, sf):
    for idx, c in con.items():
        iscale.constraint_scaling_transform(c, sf)

def set_indexed_variable_bounds(var, bounds):
    for idx, subvar in var.items():
        subvar.bounds = bounds

optarg = {
    "constr_viol_tol": 1e-8,
    "nlp_scaling_method": "user-scaling",
    "linear_solver": "ma57",
    "OF_ma57_automatic_scaling": "yes",
    "max_iter": 300,
    "tol": 1e-8,
    "halt_on_ampl_error": "no",
}

In [ ]:
operating_scenario = OperatingScenario.power_mode
t0 = time.time()
m = pyo.ConcreteModel()
m.fs = SocFlowsheet(
    dynamic=False,
    thin_electrolyte_and_oxygen_electrode=True,
    include_interconnect=True,
)
iscale.calculate_scaling_factors(m)

In [ ]:
for t in m.fs.time:
    m.fs.condenser_flash.control_volume.properties_in[t].flow_mol_phase["Liq"].domain = pyo.Reals
    m.fs.condenser_flash.control_volume.properties_in[t].flow_mol_phase["Liq"].bounds = (None, None)
    m.fs.condenser_flash.control_volume.properties_in[t].phase_frac["Liq"].domain = pyo.Reals
    m.fs.condenser_flash.control_volume.properties_in[t].phase_frac["Liq"].bounds = (None, None)
    for var in [
        m.fs.condenser_flash.control_volume.properties_in[t].log_mole_frac_tdew, 
        m.fs.condenser_flash.control_volume.properties_in[t]._mole_frac_tdew,
    ]:
        for idx in var.index_set():
            var[idx].domain = pyo.Reals
            var[idx].bounds = (None, None)

In [ ]:
if operating_scenario == OperatingScenario.power_mode:
    m.fs.initialize_build(fuel_cell_mode=True, outlvl=idaeslog.INFO, optarg=optarg)
else:
    m.fs.initialize_build(fuel_cell_mode=False, outlvl=idaeslog.INFO, optarg=optarg)

In [ ]:
for var in m.fs.manipulated_variables:
    var.unfix()
m.fs.set_performance_bounds()
m.fs.make_performance_constraints()
m.fs.cooling_water_penalty = pyo.Param(initialize=1e-5, mutable=True)
if operating_scenario == OperatingScenario.power_mode:
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2O"].fix(0.03)
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2"].fix(0.969)

    set_indexed_variable_bounds(m.fs.condenser_flash.heat_duty, (None, 0))
    set_indexed_variable_bounds(m.fs.condenser_split.split_fraction, (0.00001, 1))
    
    m.fs.feed_recycle_split.out.mole_frac_comp[0, "H2"].bounds = (0.1, 0.75)
    set_indexed_variable_bounds(m.fs.feed_recycle_split.recycle_ratio, (1e-3, 1))
    set_indexed_variable_bounds(m.fs.sweep_recycle_split.recycle_ratio, (1e-3, 1))
    set_indexed_variable_bounds(m.fs.condenser_split.recycle_ratio, (1e-3, 5))
    
    @m.fs.Constraint(m.fs.time)
    def current_density_average_limit_eqn(b, t):
        return sum([b.soc_module.solid_oxide_cell.current_density[t, iz]
                    for iz in b.soc_module.solid_oxide_cell.iznodes]
                ) / 10 == 4e3

    scale_indexed_constraint(m.fs.current_density_average_limit_eqn, 1e-3)

    # Make sure the steady-state value is nonzero so we don't have trouble with antiwindup
    # because a setpoint is right at a variable bound. We know that optimization won't
    # want a higher value of heat duty because fuel cell mode is exothermic
    m.fs.feed_heater.electric_heat_duty.fix(1e5)
    m.fs.sweep_heater.electric_heat_duty.fix(1e5)

    set_indexed_variable_bounds(m.fs.soc_module.solid_oxide_cell.current_density, (-1.3e4, 5.2e3))

    m.fs.condenser_flash.control_volume.properties_out[:].temperature.fix(273.15+50)
    set_indexed_variable_bounds(m.fs.sweep_blower.inlet.flow_mol, (500, None))
    # Surrogate objective to maximize power output while minimizing H2 mass consumption
    # Note that m.fs.total_electric_power is negative in fuel cell mode because work is
    # leaving the system. 
    m.fs.obj = pyo.Objective(
        expr=(
           1e-8 * m.fs.total_electric_power[0] + m.fs.h2_mass_consumption[0]
        )
    )
elif operating_scenario == OperatingScenario.neutral:
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2O"].fix(0.5)
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2"].fix(0.499)
    # In this idle steady state, we want net current to be zero
    @m.fs.Constraint(m.fs.time)
    def current_density_average_limit_eqn(b, t):
        return sum([b.soc_module.solid_oxide_cell.current_density[t, iz]
                    for iz in b.soc_module.solid_oxide_cell.iznodes]
                    ) / 10 == 0

    scale_indexed_constraint(m.fs.current_density_average_limit_eqn, 1e-3)

    m.fs.feed_recycle_split.recycle_ratio.fix(1)
    m.fs.sweep_recycle_split.recycle_ratio.fix(1)
    m.fs.condenser_split.recycle_ratio.fix(1)
    m.fs.sweep_blower.inlet.flow_mol.fix(4000)
    m.fs.makeup_mix.makeup.flow_mol.fix(630.5)

    m.fs.condenser_flash.control_volume.properties_out[:].temperature.fix(273.15 + 50)

    # Objective to minimize H2 and power consumption while idling.
    m.fs.obj = pyo.Objective(
        expr=(
            1e-8 * m.fs.total_electric_power[0] + m.fs.h2_mass_consumption[0]
        )
    )

else:
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2O"].fix(0.999 - 1e-14)
    m.fs.makeup_mix.makeup.mole_frac_comp[0, "H2"].fix(1e-14)
    if operating_scenario == OperatingScenario.minimum_production:
        m.fs.h2_mass_production.fix(0.4)
    elif operating_scenario == OperatingScenario.maximum_production:
        m.fs.h2_mass_production.fix(2)
    else:
        raise RuntimeError("Please choose a valid operating scenario")
    
    m.fs.feed_recycle_split.out.mole_frac_comp[0, "H2O"].bounds = (0.25, 0.4)
    set_indexed_variable_bounds(m.fs.soc_module.solid_oxide_cell.current_density, (-1.3e4, 5.2e3))
    set_indexed_variable_bounds(m.fs.sweep_blower.inlet.flow_mol, (500, None))
    # set_indexed_variable_bounds(m.fs.soc_module.solid_oxide_cell.potential, (1.18, 1.30))

    m.fs.feed_recycle_split.recycle_ratio.fix(1)
    m.fs.sweep_recycle_split.recycle_ratio.fix(1)

    m.fs.condenser_split.split_fraction[:, "recycle"].fix(0.0001)
    m.fs.condenser_split.split_fraction[:, "out"].value = 0.9999
    m.fs.condenser_flash.control_volume.properties_out[:].temperature.fix(273.15 + 50)
    # Minimize total power consumption while operating at a fixed
    # H2 production rate
    m.fs.obj = pyo.Objective(
        expr=(
            1e-8 * m.fs.total_electric_power[0]
        )
    )

@m.fs.Constraint(m.fs.time)
def sweep_concentration_eqn(b, t):
    return b.sweep_recycle_split.mixed_state[t].mole_frac_comp["O2"] <= 0.35


@m.fs.Constraint(m.fs.time)
def min_h2_feed_eqn(b, t):
    return b.feed_recycle_mix.mixed_state[t].mole_frac_comp["H2"] >= 0.05

In [ ]:
solver_obj = get_solver(options=optarg)
jac_unscaled, jac_scaled, nlp = iscale.constraint_autoscale_large_jac(m)
results = solver_obj.solve(m, tee=True)
pyo.assert_optimal_termination(results)


In [ ]:
m.fs.write_pfd(fname="soc_dynamic_flowsheet.svg")
display(
    "SOC Dynamic Flowsheet",
    SVG(filename="soc_dynamic_flowsheet.svg"),
)

In [ ]:
if operating_scenario == OperatingScenario.minimum_production:
    ms.to_json(m, fname="min_production.json.gz")
elif operating_scenario == OperatingScenario.maximum_production:
    ms.to_json(m, fname="max_production.json.gz")
elif operating_scenario == OperatingScenario.power_mode:
    ms.to_json(m, fname="power_mode.json.gz")
elif operating_scenario == OperatingScenario.neutral:
    ms.to_json(m, fname="neutral.json.gz")

In [ ]:
# for var in m.fs.manipulated_variables:
#    print(var.name + f": {var.value}")
#    var.pprint()
t1 = time.time()
print(f"Finished in {t1-t0} seconds")
print(f"Hydrogen production rate: {pyo.value(m.fs.h2_mass_production[0])} kg/s")
print(f"Cell potential: {pyo.value(m.fs.soc_module.potential_cell[0])} V")
print(f"Module total current: {pyo.value(m.fs.soc_module.total_current[0])} A")
print(f"SOC fuel outlet H2 mole frac: {pyo.value(m.fs.soc_module.fuel_outlet_mole_frac_comp_H2[0])}")
print(f"Makeup feed rate: {pyo.value(m.fs.makeup_mix.makeup.flow_mol[0])} mol/s")
#print(f"Hydrogen/water ratio: {pyo.value(m.fs.feed_hydrogen_water_ratio[0])}")
print(f"Sweep feed rate: {pyo.value(m.fs.sweep_blower.inlet.flow_mol[0])} mol/s")
print(f"Fuel-side heat duty: {pyo.value(m.fs.feed_heater.electric_heat_duty[0])} W")
print(f"Fuel-side inlet temperature: {pyo.value(m.fs.soc_module.fuel_inlet.temperature[0])} K")
# print(f"Fuel-side inlet temperature: {pyo.value(m.fs.stack_fuel_inlet_temperature[0])} K")
print(f"Fuel side outlet temperature: {pyo.value(m.fs.soc_module.fuel_outlet.temperature[0])} K")
print(f"Sweep-side heat duty: {pyo.value(m.fs.sweep_heater.electric_heat_duty[0])} W")
print(f"Sweep-side inlet temperature: {pyo.value(m.fs.soc_module.oxygen_inlet.temperature[0])} K")
# print(f"Sweep-side inlet temperature: {pyo.value(m.fs.stack_sweep_inlet_temperature[0])} K")
print(f"Oxygen side outlet temperature: {pyo.value(m.fs.soc_module.oxygen_outlet.temperature[0])} K")
print(f"Stack core temperature: {pyo.value(m.fs.stack_core_temperature[0])} K")
print(f"Fuel recycle ratio: {pyo.value(m.fs.feed_recycle_split.recycle_ratio[0])}")
print(f"Sweep recycle ratio: {pyo.value(m.fs.sweep_recycle_split.recycle_ratio[0])}")
print(f"Sweep oxygen outlet: {pyo.value(m.fs.sweep_recycle_split.mixed_state[0].mole_frac_comp['O2'])}")
print(f"Feed hydrogen inlet: {pyo.value(m.fs.feed_recycle_mix.mixed_state[0].mole_frac_comp['H2'])}")

print(f"Vent gas recirculation recycle ratio {pyo.value(m.fs.condenser_split.recycle_ratio[0])}")
# print(f"Condenser cooling water feed rate: {pyo.value(m.fs.condenser_hx.cold_side_inlet.flow_mol[0])} mol/s")
# print(f"Condenser hydrogen outlet temperature: {pyo.value(m.fs.condenser_hx.hot_side_outlet.temperature[0])} K")
print(f"Condenser heat duty: {pyo.value(m.fs.condenser_flash.heat_duty[0])} W")
print(f"Condenser hydrogen outlet temperature: {pyo.value(m.fs.condenser_flash.vap_outlet.temperature[0])} K")

for hx in [m.fs.feed_medium_exchanger, m.fs.feed_hot_exchanger, m.fs.sweep_exchanger]:
    # Test for energy conservation in heat exchangers
    Delta_E_Cold = pyo.value(
        hx.cold_side.properties[0, 0].flow_mol * hx.cold_side.properties[0, 0].enth_mol
        - hx.cold_side.properties[0, 1].flow_mol * hx.cold_side.properties[0, 1].enth_mol
    )
    Delta_E_Hot = pyo.value(
        hx.hot_side.properties[0, 0].flow_mol * hx.hot_side.properties[0, 0].enth_mol
        - hx.hot_side.properties[0, 1].flow_mol * hx.hot_side.properties[0, 1].enth_mol
    )
    assert abs(2*(Delta_E_Cold-Delta_E_Hot)/(Delta_E_Cold+Delta_E_Hot)) < 1e-3

soc = m.fs.soc_module.solid_oxide_cell

plt.figure()
plt.plot(np.array(soc.iznodes), np.array(
    [pyo.value(soc.current_density[0, i]) for i in
        soc.iznodes]) / 10, label="Current Density")
plt.plot(np.array(soc.iznodes), np.ones((10, 1)) * sum(
    [pyo.value(soc.current_density[0, i]) for i in
        soc.iznodes]) / 100, "--", label="Average Current Density")

plt.xlabel("z node", fontsize=14)
plt.ylabel(r"Current Density ($mA/cm^2$)", fontsize=14)
plt.title("Current Density", fontsize=16)

plt.figure()
plt.plot(np.array(soc.iznodes), np.array(
    [pyo.value(soc.fuel_triple_phase_boundary.potential_nernst[0, iz]
                + soc.oxygen_triple_phase_boundary.potential_nernst[0, iz])
        for iz in soc.iznodes]),
    label="Nernst Potential"
    )

plt.xlabel("z node", fontsize=14)
plt.ylabel(r"Nernst Potential (V)", fontsize=14)
plt.title("Nernst Potential", fontsize=16)

plt.figure()
plt.plot(np.array(soc.iznodes), np.array(
    [pyo.value(soc.fuel_electrode.temperature[0, 1, i]) for i in
        soc.iznodes]), label="Temperature")
plt.plot(np.array(soc.iznodes), np.ones((10, 1)) * sum(
    [pyo.value(soc.fuel_electrode.temperature[0, 1, i]) for i in
        soc.iznodes]) / 10, "--", label="Average Temperature")

plt.legend()
plt.xlabel("z node", fontsize=14)
plt.ylabel("Temperature (K)", fontsize=14)
plt.title("PEN Temperature", fontsize=16)

plt.figure()
plt.plot(np.array(soc.iznodes), np.array(
    [pyo.value(soc.fuel_electrode.dtemperature_dz[0, 1, i]) for i in
        soc.iznodes])/100)
plt.xlabel("z node", fontsize=14)
plt.ylabel(r"$\frac{dT}{dz}$ (K/cm)", fontsize=14)
plt.title("PEN Temperature Gradient", fontsize=16)

plt.show()